In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_v2_behavior()
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
from networks import observed_only_vae,IMM_vae

from utils import ones_target, zeros_target
import numpy as np
from IGNITE_model import IGNITE
import timeit
start = timeit.default_timer()
from bottleneck import push

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


Load datasets

In [4]:
data_path = 'physionet/'
with open(os.path.join(data_path, 'FP_2012_mask.pkl'), 'rb') as f:
    miss = pickle.load(f)
with open(os.path.join(data_path, 'FP_2012_LOCF.pkl'), 'rb') as f:
    LVCF = pickle.load(f)
with open(os.path.join(data_path, 'FP_2012_Intervention.pkl'), 'rb') as f:
    interventions = pickle.load(f)

In [5]:
with open(os.path.join(data_path, 'FP_2012_origional.pkl'), 'rb') as f:
    origional = pickle.load(f)
zero=np.where(np.isnan(push(origional, axis = 1)), 0 , push(origional, axis = 1))

In [6]:
with open(os.path.join(data_path, 'person.pkl'), 'rb') as f:
    mask_personalized = pickle.load(f)

pass data

In [7]:
sample_size= len(LVCF)
intervention = np.asarray(interventions)[:sample_size]
IMM_vae_x = np.multiply(LVCF[:sample_size],mask_personalized[:sample_size])
observed_only_vae_x = zero[:sample_size]
miss_sample_x = miss[:sample_size]
intervention=np.where(np.isnan(intervention), 0, intervention)

In [8]:
batch_size = 400
time_steps = observed_only_vae_x.shape[1]
num_pre_epochs = 200

shared_latent_dim = 10
z_size = shared_latent_dim
ts_feat= observed_only_vae_x.shape[2]
conditional = True
num_labels = 1

In [21]:
observed_only_vae = observed_only_vae(batch_size=batch_size, time_steps=time_steps, dim=ts_feat, z_dim=z_size,
                  conditional=conditional, num_labels=num_labels)

IMM_vae = IMM_vae(batch_size=batch_size, time_steps=time_steps, dim=ts_feat, z_dim=z_size,
                  conditional=conditional, num_labels=num_labels)


checkpoint_dir = os.path.join("data/checkpoint/")
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [22]:
exp_name = "IGNITE_dual_single_contrastive_400_treat_dot_IMM_35_001_001_0.8_discriminator"

In [13]:
tf.compat.v1.reset_default_graph()
run_config = tf.compat.v1.ConfigProto()
with tf.compat.v1.Session(config = run_config) as sess:
        model = IGNITE(sess=sess,
                      batch_size=batch_size,
                      time_steps=time_steps,
                      num_pre_epochs=num_pre_epochs,
                      checkpoint_dir=checkpoint_dir,
                      oo_vae_dim=ts_feat, 
                      z_size=z_size, observed_only_data_sample=observed_only_vae_x,
                      observed_only_vae=observed_only_vae, 
                      imm_vae_dim=ts_feat, 
                      IMM_data_sample=IMM_vae_x,
                      IMM_vae=IMM_vae,
                     binary_mask_data_sample=miss_sample_x,experiment_name=exp_name,
                      conditional=conditional, 
                       num_labels=num_labels,
                      interventions=intervention)
        model.build()
        model.train()

stop = timeit.default_timer()
print('Time: ', stop - start)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
(400, 48, 36)
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
finalize
data_size 4000
start training
VAE epoch 0
10
VAE epoch 1
10
VAE epoch 2
10
VAE epoch 3
10
VAE epoch 4
10
VAE epoch 5
10
VAE epoch 6
10
VAE epoch 7
10
VAE epoch 8
10
VAE epoch 9
10
VAE epoch 10
10
VAE epoch 11
10
VAE epoch 12
10
VAE epoch 13
10
VAE epoch 14
10
VAE epoch 15
10
VAE epoch 16
10
VAE epoch 17
10
VAE epoch 18
10
VAE epoch 19
10
VAE epoch 20
10
VAE epoch 21
10
VAE epoch 22
10
VAE epoch 23
10
VAE epoch 24
10
VAE epoch 25
10
VAE epoch 26
10
VAE epoch 27
10
VAE epoch 28
10
VAE epoch 29
10
VAE epoch 30
10
VAE epoch 31
10
VAE epoch 32
10
VAE epoch 33
10
VAE epoch 34
10
VAE epoch 35
10
VAE epoch 36
10
VAE epoch 37
10
VAE epoch 38
10
VAE epoch 39
10
VAE epoch 40
10
VAE epoch 41
10
VAE epoch 42
10
VAE epoch 43
10
VAE epoch 44
10
VAE epoch 